In [1]:
import os
import pandas as pd

folder_path = '.\QQQ'

options_dfs = []

for filename in os.listdir(folder_path):
    
    file_path = os.path.join(folder_path, filename)
  
    options_df = pd.read_csv(file_path, low_memory=False)
    
    options_dfs.append(options_df)

options_df = pd.concat(options_dfs, ignore_index=True)  
  
# fix column names by removing [, ], " " characters
options_df.columns = options_df.columns.str.strip('[] ')

# Define data types for all columns
for col in options_df.columns:
    if col.startswith('C_') or col.startswith('P_'):
        options_df[col] = pd.to_numeric(options_df[col], errors='coerce')

dtypes = {
    'QUOTE_UNIXTIME': 'int64',
    'QUOTE_READTIME': 'datetime64[ns]',
    'QUOTE_DATE': 'datetime64[ns]',
    'QUOTE_TIME_HOURS': 'float64',
    'UNDERLYING_LAST': 'float64',
    'EXPIRE_DATE': 'datetime64[ns]',
    'EXPIRE_UNIX': 'int64',
    'DTE': 'float64',
    'C_DELTA': 'float64',
    'C_GAMMA': 'float64',
    'C_VEGA': 'float64',
    'C_THETA': 'float64',
    'C_RHO': 'float64',
    'C_IV': 'float64',
    'C_VOLUME': 'float64',
    'C_LAST': 'float64',
    'C_SIZE': 'float64',
    'C_BID': 'float64',
    'C_ASK': 'float64',
    'STRIKE': 'float64',
    'P_BID': 'float64',
    'P_ASK': 'float64',
    'P_SIZE': 'float64',
    'P_LAST': 'float64',
    'P_DELTA': 'float64',
    'P_VOLUME': 'float64',
    'STRIKE_DISTANCE': 'float64',
    'STRIKE_DISTANCE_PCT': 'float64'
}

# Convert columns to the specified data types
options_df = options_df.astype(dtypes)

C:\Users\gabri\AppData\Local\Temp\ipykernel_15076\3245454215.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
filtered_options_df = options_df[(options_df["P_VOLUME"]>= 100) & (options_df["C_VOLUME"] >= 100)]
filtered_options_df = filtered_options_df[['QUOTE_DATE', 'EXPIRE_DATE', 'UNDERLYING_LAST', 'C_VOLUME', 'C_LAST', 'C_BID', 'C_ASK', 'STRIKE', 'P_VOLUME', 'P_LAST', 'P_BID', 'P_ASK']]
filtered_options_df = filtered_options_df.reset_index(drop=True)
filtered_options_df['SC'] = (filtered_options_df['C_BID'] + filtered_options_df['STRIKE'] - filtered_options_df['P_ASK'] - filtered_options_df['UNDERLYING_LAST']) * 100
filtered_options_df['BC'] = (filtered_options_df['P_BID'] - filtered_options_df['STRIKE'] - filtered_options_df['C_ASK'] + filtered_options_df['UNDERLYING_LAST']) * 100
filtered_options_df

,QUOTE_DATE,EXPIRE_DATE,UNDERLYING_LAST,C_VOLUME,C_LAST,C_BID,C_ASK,STRIKE,P_VOLUME,P_LAST,P_BID,P_ASK,SC,BC
0,2023-01-03,2023-01-03,264.46,213.0,6.27,6.32,6.65,258.00,3933.0,0.01,0.00,0.01,-1.500000e+01,-19.0
1,2023-01-03,2023-01-03,264.46,1265.0,5.00,4.42,4.60,260.00,10768.0,0.01,0.00,0.01,-5.000000e+00,-14.0
2,2023-01-03,2023-01-03,264.46,759.0,3.41,3.48,3.67,261.00,7243.0,0.01,0.00,0.01,1.000000e+00,-21.0
3,2023-01-03,2023-01-03,264.46,4309.0,2.54,2.44,2.64,262.00,21478.0,0.01,0.00,0.01,-3.000000e+00,-18.0
4,2023-01-03,2023-01-03,264.46,10114.0,1.65,1.48,1.63,263.00,23798.0,0.02,0.01,0.02,5.684342e-12,-16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30831,2023-12-29,2024-02-16,409.50,160.0,10.40,10.30,10.35,410.78,125.0,8.65,8.84,8.89,2.690000e+02,-279.0
30832,2023-12-29,2024-03-15,409.50,1142.0,18.55,18.25,18.35,404.78,481.0,8.77,8.80,8.84,4.690000e+02,-483.0
30833,2023-12-29,2024-03-15,409.50,120.0,15.01,14.72,14.81,409.78,233.0,10.56,10.57,10.63,4.370000e+02,-452.0
30834,2023-12-29,2024-06-21,409.50,145.0,37.78,35.68,35.91,394.78,489.0,12.03,12.06,12.14,8.820000e+02,-913.0


In [10]:
def max_or_zero(x):
    max_val = x.max()
    return max_val if max_val >= 0 else 0

max_SC = filtered_options_df.groupby(filtered_options_df['QUOTE_DATE'].dt.date)['SC'].apply(max_or_zero)
max_BC = filtered_options_df.groupby(filtered_options_df['QUOTE_DATE'].dt.date)['BC'].apply(max_or_zero)

options_df_by_date = pd.DataFrame({
    'max_SC': max_SC,
    'max_BC': max_BC
})

options_df_by_date.reset_index(inplace=True)
options_df_by_date["max_deal"] = pd.concat([options_df_by_date['max_SC'], options_df_by_date['max_BC']], axis=1).max(axis=1)

print("The sum is: ", options_df_by_date['max_deal'].sum())


The sum is:  213908.0


In [11]:
top5_rows = options_df_by_date.nlargest(5, "max_deal")
matched_rows = []

for index, row in top5_rows.iterrows():
    new_rows = filtered_options_df[(filtered_options_df["SC"] == row['max_deal']) | (filtered_options_df["BC"] == row["max_deal"])]
    matched_rows.append(new_rows)

matched_rows_df = pd.concat(matched_rows, ignore_index=True)

print(matched_rows_df)


  QUOTE_DATE EXPIRE_DATE  UNDERLYING_LAST  C_VOLUME  C_LAST  C_BID  C_ASK  \
0 2023-12-12  2025-06-20           398.75    1251.0   61.33  60.52  61.95   
1 2023-08-07  2024-12-20           375.18     370.0   58.98  59.56  60.05   
2 2023-08-08  2024-12-20           371.94     133.0   55.59  57.23  57.92   
3 2023-05-31  2024-12-20           347.66     250.0   36.30  35.80  36.43   
4 2023-07-06  2024-09-20           367.47    2000.0   35.80  35.73  36.10   

   STRIKE  P_VOLUME  P_LAST  P_BID  P_ASK      SC      BC  
0   385.0    1251.0   25.71  24.96  25.59  2118.0 -2324.0  
1   360.0     463.0   24.45  23.97  24.13  2025.0 -2090.0  
2   360.0     177.0   26.59  25.03  25.38  1991.0 -2095.0  
3   375.0     250.0   43.50  43.29  43.83  1931.0 -2048.0  
4   385.0    2000.0   33.98  33.81  34.11  1915.0 -1982.0  
